In [9]:
import numpy as np
import pandas as pd
from numpy import random
import sqlite3

import re

import random

import warnings
warnings.filterwarnings('ignore')

seed = 1998

In [10]:
area_list = ['Dobong', 'Dongdaemun', 'Dongjag', 'eunpyeong', 'geumcheon',
'Guro', 'GwanUck', 'Jongno', 'Junggu', 'Jungnang', 'Kongbuk',
'Kongdong', 'Kongjin', 'Kongnam', 'Kongseo', 'Mapo', 'Nowon', 'Seocho',
'Seodaemun', 'Seongbuk', 'Seongdong', 'songpa', 'Yangcheon',
'Yeongdeungpo', 'Yongsan']

len(area_list)

25

In [11]:
def solve(x):
    a = re.findall(r'\d{4}년 \d{1,2}월 \d{1,2}일', str(x))
    try: return a[0]
    except: return np.NAN

data = pd.DataFrame()

for area in area_list:
    tem = pd.read_csv(f'./user_csv/{area}.csv')
    tem['userid'] = tem['link'].apply(lambda x: str(re.findall(r'my/[a-z0-9]+/', x))[5:-3])
    tem['date'] = tem['user'].apply(lambda x: solve(x))

    tem = tem[['userid', 'rest', 'date']]
    tem = tem.drop_duplicates()
    data = pd.concat([data, tem])

data = data.drop_duplicates() # 30만개정도 빠짐
data = data[~data['rest'].isnull()] # 7개정도 null 값이 있더라
data.reset_index(drop=True, inplace = True)

In [13]:
rest = pd.read_csv('data/rest.csv')
rest = rest[['url', 'rest_code']]

user = pd.read_csv('data/user.csv')
user = user[['user','user_code']]

data = data[['userid', 'rest', 'date']]
data = data.drop_duplicates(['userid','rest'])

In [14]:
data = pd.merge(data, rest, left_on = 'rest', right_on = 'url', how = 'left')
data = data[~data['rest_code'].isnull()]
data['rest_code'] = data['rest_code'].astype('int')
data = pd.merge(data, user, left_on = 'userid', right_on = 'user', how = 'left')
data['user_code'] = data['user_code'].fillna(-1)
data['user_code'] = data['user_code'].astype('int')
data = data.drop_duplicates(['userid','rest'])
data = data[['userid', 'rest', 'date', 'rest_code', 'user_code']].reset_index(drop = True)

In [16]:
data.to_csv('../airflow/data/data_all.csv', index = False)
data.to_csv('data/data_all.csv', index = False)